In [ ]:
import json
import time
from google import genai
import os
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Configuration constants
SERVICE_ACCOUNT_FILE = "credentials.json"
SPREADSHEET_ID = "1NywRL9IBR69R0eSrOE9T6mVUbfJHwaALL0vp2K0TLbY"
INPUT_RANGE = "main!A:G"  # Extended to include all three classification columns
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
DELAY_BETWEEN_REQUESTS = 1.0

# Classification taxonomies
SOURCE_TYPES = [
    "News Outlet",
    "Think Tank",
    "Government Body",
    "Academic Institution",
    "Advocacy Organization",
    "Research Institute",
    "Fact Checker",
    "Legal Organization",
    "International Organization",
    "Investigative Outlet",
    "Trade/Industry Publication",
    "NGO / Civil Society",
    "Media Watchdog",
    "Government Contractor / FFRDC",
]

TOPIC_TAXONOMY = [
    "General News",
    "Politics & Government",
    "Foreign Policy & Diplomacy",
    "Military & Defense",
    "Intelligence & National Security",
    "Legal & Judicial",
    "Criminal Justice",
    "Democracy & Elections",
    "Human Rights",
    "Immigration & Migration",
    "Tax & Fiscal Policy",
    "Economics & Labor",
    "Business & Finance",
    "Science & Technology",
    "Health & Medicine",
    "Energy & Environment",
    "Education",
    "Media & Disinformation",
    "Conflict & Peace Studies",
    "Organized Crime & Corruption",
    "Legislative / Congressional",
]

GEOGRAPHIC_FOCUS = [
    "Global",
    "United States",
    "Europe",
    "Middle East",
    "Africa",
    "Asia-Pacific",
    "Latin America",
    "Russia & Eurasia",
    "Washington D.C.",
]


def process_primary_topic():
    """
    Main function that handles the entire source classification process.
    Reads organization data from Google Sheets, classifies each organization 
    across three dimensions (source_type, primary_topic, geographic_focus) 
    using Google Gemini API, and writes results back to the sheet.
    """
    
    # Get API key
    api_key = input("🔑 Enter your Gemini API key: ").strip()
    
    if not api_key:
        print("❌ Error: GEMINI_API_KEY environment variable not set")
        print("\nTo set it:")
        print("  Linux/Mac: export GEMINI_API_KEY='your-api-key'")
        print("  Windows: set GEMINI_API_KEY=your-api-key")
        return
    
    # Initialize Gemini client
    client = genai.Client(api_key=api_key)
    
    # Set up Google Sheets connection
    print("🔗 Connecting to Google Sheets...")
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, 
        scopes=SCOPES
    )
    sheets_service = build("sheets", "v4", credentials=creds)
    print("✅ Connected to Google Sheets")
    
    # Read data from the spreadsheet
    print(f"📂 Loading data from Google Sheet...")
    sheet = sheets_service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID, 
        range=INPUT_RANGE
    ).execute()
    
    values = result.get("values", [])
    
    if not values:
        print("❌ No data found in sheet")
        return
    
    # Process the sheet data
    headers = values[0]
    data = []
    for i, row in enumerate(values[1:], start=1):
        row_data = row + [''] * (len(headers) - len(row))
        row_dict = {headers[j]: row_data[j] for j in range(len(headers))}
        row_dict['_row_index'] = i + 1
        data.append(row_dict)
    
    if not data:
        print("❌ No data to process")
        return
    
    print(f"✅ Loaded {len(data)} sources")
    
    # Show current status
    already_filled = sum(1 for row in data 
                        if row.get('source_type', '').strip() 
                        and row.get('primary_topic', '').strip() 
                        and row.get('geographic_focus', '').strip())
    needs_research = len(data) - already_filled
    
    print(f"📊 Status: {already_filled} already classified, {needs_research} need research")
    print(f"🚀 Starting classification...\n")
    
    # Process each row
    start_time = time.time()
    updated_count = 0
    skipped_count = 0
    
    for idx, row in enumerate(data):
        name = row.get('name', '').strip()
        url = row.get('url', '').strip()
        existing_source_type = row.get('source_type', '').strip()
        existing_topic = row.get('primary_topic', '').strip()
        existing_geo = row.get('geographic_focus', '').strip()
        row_index = row.get('_row_index')
        
        # Skip rows with missing data
        if not name or not url:
            print(f"[{idx + 1}/{len(data)}] Skipping row {row_index}: missing name or URL")
            continue
        
        # Skip rows that already have all classifications
        if existing_source_type and existing_topic and existing_geo:
            print(f"[{idx + 1}/{len(data)}] Skipping {name}: already classified")
            skipped_count += 1
            continue
        
        print(f"[{idx + 1}/{len(data)}] Classifying: {name}")
        
        # Classify the organization using Google Gemini API
        try:
            prompt = f"""Classify the source "{name}" ({url}) using exactly the values provided.

SOURCE_TYPES: {json.dumps(SOURCE_TYPES)}
TOPIC_TAXONOMY: {json.dumps(TOPIC_TAXONOMY)}
GEOGRAPHIC_FOCUS: {json.dumps(GEOGRAPHIC_FOCUS)}

Respond ONLY with valid JSON, no other text:
{{
  "source_type": "exact value from SOURCE_TYPES",
  "primary_topic": "exact value from TOPIC_TAXONOMY",
  "geographic_focus": "exact value from GEOGRAPHIC_FOCUS",
  "confidence": "high, medium, or low",
  "brief_reasoning": "one sentence"
}}

DO NOT include any text outside the JSON. DO NOT use markdown code blocks."""

            response = client.models.generate_content(
                model="gemini-3-flash-preview",
                contents=prompt
            )
            
            response_text = response.text.strip()
            response_text = response_text.replace("```json", "").replace("```", "").strip()
            result = json.loads(response_text)
            
            # Validate that returned values are from the controlled vocabularies
            if result['source_type'] not in SOURCE_TYPES:
                print(f"  ⚠️  Warning: Invalid source_type '{result['source_type']}', skipping")
                continue
            if result['primary_topic'] not in TOPIC_TAXONOMY:
                print(f"  ⚠️  Warning: Invalid primary_topic '{result['primary_topic']}', skipping")
                continue
            if result['geographic_focus'] not in GEOGRAPHIC_FOCUS:
                print(f"  ⚠️  Warning: Invalid geographic_focus '{result['geographic_focus']}', skipping")
                continue
            
            # Write all three results back to the spreadsheet
            # Assuming columns: A=name, B=url, C=profit-status, D=primary_topic, E=source_type, F=geographic_focus
            updates = [
                {
                    'range': f"main!D{row_index}",
                    'values': [[result['primary_topic']]]
                },
                {
                    'range': f"main!E{row_index}",
                    'values': [[result['source_type']]]
                },
                {
                    'range': f"main!F{row_index}",
                    'values': [[result['geographic_focus']]]
                }
            ]
            
            batch_update_body = {
                'valueInputOption': 'RAW',
                'data': updates
            }
            
            sheets_service.spreadsheets().values().batchUpdate(
                spreadsheetId=SPREADSHEET_ID,
                body=batch_update_body
            ).execute()
            
            updated_count += 1
            print(f"  ✓ {result['source_type']} | {result['primary_topic']} | {result['geographic_focus']}")
            print(f"    Confidence: {result['confidence']} - {result['brief_reasoning']}")
            
        except json.JSONDecodeError as e:
            print(f"  ✗ Failed to parse JSON: {str(e)}")
            print(f"    Response was: {response_text[:200]}...")
        except Exception as e:
            print(f"  ✗ Failed to classify: {str(e)}")
        
        # Rate limiting
        time.sleep(DELAY_BETWEEN_REQUESTS)
    
    # Summary
    elapsed = time.time() - start_time
    print(f"\n✅ Complete! Processed {len(data)} sources in {elapsed/60:.1f} minutes")
    print(f"📝 Updated {updated_count} rows in Google Sheet")
    print(f"⏭️  Skipped {skipped_count} rows (already classified)")


if __name__ == "__main__":
    process_primary_topic()

🔗 Connecting to Google Sheets...
✅ Connected to Google Sheets
📂 Loading data from Google Sheet...
✅ Loaded 364 sources
📊 Status: 0 already classified, 364 need research
🚀 Starting classification...

[1/364] Classifying: Insight Crime
  ✓ Investigative Outlet | Organized Crime & Corruption | Latin America
    Confidence: high - InSight Crime is a non-profit organization dedicated to the study and investigative reporting of organized crime and citizen security in the Americas.
[2/364] Classifying: United States Institute of Peace
  ✓ Research Institute | Conflict & Peace Studies | Global
    Confidence: high - USIP is a federally funded research organization dedicated to the study and prevention of violent international conflict.
[3/364] Classifying: Pew Research
  ✓ Research Institute | General News | Global
    Confidence: high - Pew Research Center is a nonpartisan research institute conducting public opinion polling and social science research on diverse issues globally.
[4/364] Clas